# Imports and preparation

In [1]:
import os

In [2]:
import numpy as np

In [3]:
import pandas as pd

In [4]:
from tqdm import tqdm

In [21]:
import time, gc
from haystack import Pipeline
from haystack.pipelines import DocumentSearchPipeline

In [38]:
from util import util_elastic_search

In [ ]:
from util import util_search

In [ ]:
import importlib
importlib.reload(util_search)

In [ ]:
%%time
from haystack.document_stores import ElasticsearchDocumentStore
from haystack.nodes import EmbeddingRetriever, BM25Retriever, MonoT5RankerLimit, SentenceTransformersRankerLimit
import logging
logging.getLogger("haystack").setLevel(logging.WARNING) #WARNING, INFO

CPU times: user 224 µs, sys: 0 ns, total: 224 µs
Wall time: 226 µs


In [5]:
# Para ter repetibilidade nos resultados
random_state = 1

# Tratar valores infinitos (+ e -) como np.NaN
pd.options.mode.use_inf_as_na = True

# IMPORTANTE para tornar figuras interativas
# %matplotlib notebook

# Tamanho padrão das figuras
figsize=(10,6)

pd.set_option('display.max_row', 1000)

pd.set_option('display.max_columns', 50)

pd.set_option('display.column_space', 40)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.width', 200)


In [9]:
PATH_QUERY = '../data/index_juris/query.csv'
PATH_QREL =  '../data/index_juris/qrel.csv'

In [11]:
PATH_SEARCH_EXPERIMENT =  '../data/search/index_juris/search_experiment.csv'
PATH_SEARCH_RESULT =  '../data/search/index_juris/search_experiment_result.csv'

# Create reference to index (elastic search)

Documentation https://docs.haystack.deepset.ai/docs
https://haystack.deepset.ai/tutorials/09_dpr_training 


In [39]:
index_dict = util_elastic_search.return_indexes('indir', parm_print=True)

Index: indir_index_juris
{'health': 'yellow', 'status': 'open', 'index': 'indir_index_juris', 'uuid': 'M-G39ddyR_ijjhxzekyssw', 'pri': '1', 'rep': '1', 'docs.count': '0', 'docs.deleted': '0', 'store.size': '208b', 'pri.store.size': '208b'}

Index: indir_index_juris_vce
{'health': 'yellow', 'status': 'open', 'index': 'indir_index_juris_vce', 'uuid': 'kFcEPIh_Spe7bqJPAfqUtg', 'pri': '1', 'rep': '1', 'docs.count': '13252', 'docs.deleted': '13252', 'store.size': '289.2mb', 'pri.store.size': '289.2mb'}

There are no index with name indir%


In [32]:
INDEX_NAME = 'indir_index_juris'

In [34]:
document_store_index = ElasticsearchDocumentStore(
        host='localhost',
        username='', password='',
        index=INDEX_NAME,
        similarity='dot_product',
        search_fields="content",
        content_field = "content",
        embedding_field = "embedding",
        embedding_dim = 1024,
        # excluded_meta_data=['embedding'], # that should not be returned 
        duplicate_documents='fail',
        return_embedding=False,
    )

In [35]:
print(f"\nQtd de documentos {document_store.get_document_count()}")
print(f"\nQtd de embeddings {document_store.get_embedding_count()}")
print(f"\nDocumento.id=1: {document_store.get_document_by_id('1')}")



Qtd de documentos 0

Qtd de embeddings 0

Documento.id=1: None


In [19]:
nome_modelo_embedding_model_sts_rufimelo = "rufimelo/Legal-BERTimbau-sts-large-ma-v3"
# requer Embedding dimensions of the model (1024) don't match the embedding dimensions of the document store (768). Initiate ElasticsearchDocumentStore again with arg embedding_dim=1024.
# nome_modelo = "unicamp-dl/ptt5-base-pt-msmarco-100k-v2"
# nome_modelo = "pierreguillou/bert-base-cased-squad-v1.1-portuguese"

nome_caminho_modelo = "/home/borela/fontes/relevar-busca/modelo/" + nome_modelo_embedding_model_sts_rufimelo

assert os.path.exists(nome_caminho_modelo), f"Path para {nome_caminho_modelo} não existe!"


In [26]:
retriever_sts = EmbeddingRetriever(
    document_store=document_store,
    embedding_model=nome_caminho_modelo,
    model_format="sentence_transformers",
    pooling_strategy = 'cls_token',
    progress_bar = False
)


In [ ]:
# Test query
parm_query = "trata-se de uma denúncia contra o prefeito de Manhuaçu por não haver pago os funcionários da área de limpeza urbana"

# defining retriever BM25
retriever_bm25 = BM25Retriever(document_store=document_store_bm25,all_terms_must_match=False)
pipeline_bm25 = DocumentSearchPipeline(retriever_bm25)
doc_search_return = pipeline_bm25.run(
    query=parm_query,
    params={
        "Retriever": {
            "top_k": 10,
            "filters":{"class": ["Termo"]}
        }
    }
)
document = doc_search_return["documents"][0]
print(document.id, document.content, document.meta, document.score, document.embedding)
util_search.detail_document_found(doc_search_return)
print([docto.id for docto in doc_search_return['documents']])

In [ ]:
## defining retriever STS (sentence similarity)
nome_modelo_embedding_model_sts_rufimelo = "rufimelo/Legal-BERTimbau-sts-large-ma-v3"
# requer Embedding dimensions of the model (1024) don't match the embedding dimensions of the document store (768). Initiate ElasticsearchDocumentStore again with arg embedding_dim=1024.
# nome_modelo = "unicamp-dl/ptt5-base-pt-msmarco-100k-v2"
# nome_modelo = "pierreguillou/bert-base-cased-squad-v1.1-portuguese"
nome_caminho_modelo = "/home/borela/fontes/relevar-busca/modelo/" + nome_modelo_embedding_model_sts_rufimelo
assert os.path.exists(nome_caminho_modelo), f"Path para {nome_caminho_modelo} não existe!"

retriever_sts = EmbeddingRetriever(
    document_store=document_store_er_1024,
   embedding_model=nome_caminho_modelo,
   model_format="sentence_transformers",
    # pooling_strategy = 'cls_token',
    progress_bar = False
)

pipeline_sts = DocumentSearchPipeline(retriever_sts)
doc_search_return = pipeline_sts.run(
    query=parm_query,
    params={
        "Retriever": {
            "top_k": 10,
            "filters":{"class": ["Termo"]}
        }
    }
)
document = doc_search_return["documents"][0]
print(document.id, document.content, document.meta, document.score, document.embedding)
util_search.detail_document_found(doc_search_return)
print([docto.id for docto in doc_search_return['documents']])


In [ ]:
raise Exception ('Stop execution')

# Data load

## Query data load

In [ ]:
df_query = pd.read_csv(PATH_QUERY)

In [ ]:
df_query.shape

(16022, 11)

In [ ]:
df_query.head()

,ID,TEXT,REFERENCE_LIST,PARADIGMATIC,AREA_NAME,AREA_ID_DESCRIPTOR,NORMATIVE_PROCESS_TYPE,NORMATIVE_IDENTIFICATION,NORMATIVE_DATE,NORMATIVE_AUTHOR_TYPE,NORMATIVE_AUTHOR_NAME
0,34899,A transferência de documentos da entidade para local impróprio ao armazename...,"Lei Ordinária 8.443/1992, art. 58, inciso II",NaN,Responsabilidade,775,REPRESENTAÇÃO,Acórdão 2669/2012 - Plenário,2012-10-03,RELATOR,JOSÉ JORGE
1,30271,"A contratação de médicos e profissionais da área de saúde, como colaboradore...",NaN,NaN,Pessoal,1131,REPRESENTAÇÃO,Acórdão 2669/2012 - Plenário,2012-10-03,RELATOR,JOSÉ JORGE
2,26574,"Para que seja conhecido o recurso de revisão, não basta apenas que se aprese...",NaN,NaN,Direito processual,5288,TOMADA DE CONTAS,Acórdão 514/2013 - Plenário,2013-03-13,RELATOR,ANA ARRAES
3,17902,A contratação de serviços comuns de engenharia que possam ser objetivamente ...,NaN,NaN,Licitação,932,RELATÓRIO DE LEVANTAMENTO,Acórdão 3144/2012 - Plenário,2012-11-21,RELATOR,ANA ARRAES
4,26089,"A Fundação Banco do Brasil, por receber recursos da União, deve observar, qu...",NaN,NaN,Competência do TCU,5095,TOMADA DE CONTAS ESPECIAL,Acórdão 2071/2013 - Plenário,2013-08-07,RELATOR,JOSÉ JORGE


## Qrel data load

In [22]:
df_qrel = pd.read_csv(PATH_QREL)

In [23]:
df_qrel.shape

(94653, 3)

In [24]:
df_qrel.head()

,ID_QUERY,ID_DOCTO,TYPE
0,158,15147,INDEXACAO_EXTRA
1,37,15147,INDEXACAO_EXTRA
2,178,15147,INDEXACAO_EXTRA
3,14564,15147,INDEXACAO_EXTRA
4,9219,15147,INDEXACAO_EXTRA
